In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=b309d82dc8410b043c1c7ba4ca31029aeb500bb377793368fcc244ecc3186e69
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


In [ ]:
import numpy as np
import pandas as pd
import emoji
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the data
mapping = pd.read_csv("/content/drive/MyDrive/internship/Mapping.csv")
output = pd.read_csv("/content/drive/MyDrive/internship/Test.csv")
train = pd.read_csv("/content/drive/MyDrive/internship/Train.csv")
test = pd.read_csv("/content/drive/MyDrive/internship/OutputFormat.csv")

In [ ]:
train

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3
...,...,...,...
69995,69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,69997,If one of my daughters is wearing this and ask...,2
69998,69998,Guess who whoop people on THEIR homecoming?! #...,3


In [ ]:
#subset train DataFrame
subset_train=train.sample(frac=0.3).reset_index(drop=True)
subset_train.shape

(21000, 3)

In [ ]:
train=subset_train

In [ ]:
train

,Unnamed: 0,TEXT,Label
0,59001,Show #8 tonight. Toronto let's get it! @user #...,0
1,55604,Times Square #ny #newyork #usa #fisheyelens #t...,11
2,68973,"#SundayFunday with my boos @ Third Ward, Houst...",8
3,42322,F R I E N D S @ West Chester University of PA\n,14
4,28096,We could say that meditation doesn't have a re...,13
...,...,...,...
20995,47845,"All I want for Christmas is you @ Avon Lake, O...",5
20996,31448,See... What had happened was... (@ Wally's Mil...,0
20997,44761,Snapbacks and Cody #Cody #pups @ Westlake Golf...,9
20998,20078,Loved having our family together for Thanksgiv...,9


In [ ]:
# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['TEXT'])
vocab_size = len(tokenizer.word_index) + 1
train_sequences = tokenizer.texts_to_sequences(train['TEXT'])
test_sequences = tokenizer.texts_to_sequences(train['TEXT'])
max_seq_length = 100  # define the maximum sequence length
train_data = pad_sequences(train_sequences, maxlen=max_seq_length)
test_data = pad_sequences(test_sequences, maxlen=max_seq_length)


In [ ]:
# Check unique values in the label column
unique_labels = train['Label'].unique()
num_classes = len(unique_labels)


In [ ]:
# Preprocess labels
label_mapping = {label: index for index, label in enumerate(unique_labels)}
train_labels = train['Label'].map(label_mapping)
train_labels = to_categorical(train_labels, num_classes=num_classes)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))


# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=10, batch_size=32)


Epoch 1/10
657/657 [==============================] - 147s 219ms/step - loss: 2.5229 - accuracy: 0.2500
Epoch 2/10
657/657 [==============================] - 138s 210ms/step - loss: 1.9778 - accuracy: 0.4077
Epoch 3/10
657/657 [==============================] - 144s 219ms/step - loss: 1.2303 - accuracy: 0.6304
Epoch 4/10
657/657 [==============================] - 140s 214ms/step - loss: 0.6447 - accuracy: 0.8133
Epoch 5/10
657/657 [==============================] - 139s 212ms/step - loss: 0.3403 - accuracy: 0.9013
Epoch 6/10
657/657 [==============================] - 141s 215ms/step - loss: 0.1976 - accuracy: 0.9431
Epoch 7/10
657/657 [==============================] - 144s 220ms/step - loss: 0.1343 - accuracy: 0.9608
Epoch 8/10
657/657 [==============================] - 137s 208ms/step - loss: 0.0954 - accuracy: 0.9722
Epoch 9/10
657/657 [==============================] - 140s 213ms/step - loss: 0.0743 - accuracy: 0.9785
Epoch 10/10
657/657 [==============================] - 140s 213m

In [ ]:
# Define a function for predicting emojis
def predict_emoji(input_sentence):
    # Preprocess the input sentence
    sequence = tokenizer.texts_to_sequences([input_sentence])
    data = pad_sequences(sequence, maxlen=max_seq_length)

    # Make prediction
    prediction = model.predict(data)
    predicted_label = np.argmax(prediction)

    # Map the predicted label to the emoji
    predicted_emoji = mapping['emoticons'][predicted_label]

    return predicted_emoji


In [ ]:
user_input = "i love deep learing "
predicted_emoji = predict_emoji(user_input)
print(f"Input Sentence: {user_input}")
print(f"Predicted Emoji: {predicted_emoji}")

1/1 [==============================] - 1s 549ms/step
Input Sentence: i love deep learing 
Predicted Emoji: 💕
